In [2]:
import numpy as np
import dataset
from defense import *
from attack import RF
from joblib import Parallel, delayed

In [6]:
from pathlib import Path
import json
data = np.load("data/extracted/gpts.npz", allow_pickle=True)
traces, labels = data["traces"], data["labels"]
root = Path("data/wang/gpts")
label2idx = json.load(open("data/wflibdata/gpts_full_info.json"))
label_count_dict = {}
for i in range(len(traces)):
    trace, label = traces[i], labels[i]
    if label in label_count_dict:
        idx = label_count_dict[label]
        label_count_dict[label] += 1
    else:
        idx = 0
        label_count_dict[label] = 1

    trace_file = root / f"{label2idx[label]}-{idx}"
    with open(trace_file, "w") as f:
        trace[:, 0] = trace[:, 0] - trace[0, 0]
        for cell in trace:
            f.write(f"{float(cell[0]):.5f}\t{int(cell[1]*cell[2])}\n")

In [4]:
ds = dataset.get_ds("df")
for defense in [
    Empty(),
    Front({"client_dummy_pkt_num": 4250, "server_dummy_pkt_num": 4250}, name="front_4250"),
    Tamaraw({"client_interval": 0.02, "server_interval": 0.006}, name="tamaraw_2_6"),
    Wfgan({"tol": 0.6}, name="gan300_tol6"),
    RegulaTor(
        param={
            "orig_rate": 350,
            "depreciation_rate": 0.94,
            "burst_threshold": 3.55,
            "upload_ratio": 2.6,
            "delay_cap": 1.77,
        }
    ),
]:
    ds.load_defended(defense, parallel=100)
    ds.to_wang_format(defense)
ds.load_defended(
    Pred({"security": 0.7, "data_vs_time": 2}, name="pred300_security_7_data_vs_time_20"),
    parallel=100,
)
ds.to_wang_format("pred300_security_7_data_vs_time_20")

regulator4 [0.23081952414915408, 0.6994640797567464]


In [ ]:
import attack as wfpattack
from config import *
import numpy as np
import dataset
from defense import *
from attack import RF
from joblib import Parallel, delayed


def get_cache(name, scenario, attack, defense):
    ds = dataset.get_ds(name, scenario=scenario)
    ds.load_defended(defense)
    attack = wfpattack.get_attack(attack)
    ds.get_cached_data(attack(10000, ds.num_classes(), 2))


defenses = []
for config in configs:
    defenses.append(Customized(config["client"], config["server"], config["name"], mode="moderate"))
for config in configs:
    defenses.append(
        Customized(config["client"], config["server"], config["name"], mode="moderate25")
    )

for name in ["ours", "df"]:
    for scenario in ["open-world", "closed-world"]:
        ds = dataset.get_ds(name, scenario=scenario)
        for attack in ["RF", "DF"]:
            attack = wfpattack.get_attack(attack)
            for defense in defenses:
                ds.load_defended(defense)
                ds.get_cached_data(attack(10000, ds.num_classes(), 2, n_jobs=60))
# Parallel(n_jobs=5)(delayed(get_cache)(name) for name in ["ours"])